<a href="https://colab.research.google.com/github/dasom222g/learn-LLM/blob/main/05_Transformer(chatbot).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Transformer

## Transformer 챗봇



트랜스포머 모델을 활용해 간단한 챗봇을 구축하겠습니다.
챗봇은 입력 시퀀스가 들어왔을 때 다른 시퀀스를 출력하는 seq2seq 모델이기 때문에 실습은 다음과 같은 순서로 진행해야 합니다.


(1) 전처리
  - 질문, 답변문 토큰화
  - 단어 사전 구축
  - 토큰 길이 맞추기(패딩 추가)
  - 인코딩
  - 데이터셋, 데이터로더 구축

(2) 모델 구축(트랜스포머)
  - 포지셔널 인코딩 함수
  - 마스킹 함수
  - 모델 설계

(3) 모델 하이퍼파라미터 설정
  - 손실 함수
  - 옵티마이저
  - 트랜스포머 하이퍼파라미터

(4) 모델 학습
  - epoch을 돌리며 학습과 검증

(5) 모델 평가
  - 학습한 모델 평가


In [ ]:
# 필요한 라이브러리 불러오기

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import urllib.request

import numpy as np
import pandas as pd

from collections import Counter
import time
import matplotlib.pyplot as plt

# 토큰화를 위한 토크나이저
from transformers import BertTokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
# 실습에 활용할 데이터 다운로드

urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


### 1) 전처리

질문과 답변 쌍으로 구성된 데이터를 준비했습니다. 이제 seq2seq 모델에 통과할 수 있는 형태로 전처리를 진행하겠습니다.

전처리는 다음과 같은 작업을 진행해야 합니다.

- 질문, 답변문 토큰화
- 단어 사전 구축
- 토큰 길이 맞추기(패딩 추가)
- 인코딩
- 데이터셋, 데이터로더 구축

먼저 질문과 답변문의 토큰화 작업을 진행하겠습니다. 불러온 토크나이저를 활용해 문장을 토큰 단위로 나눠줍니다.

In [ ]:
# 질문 토큰화

def preprocess_src(text):
    tokens = tokenizer.tokenize(text)
    return tokens

In [ ]:
# 답변 토큰화
def preprocess_trg_in(text):
  text = ['<sos>'] + tokenizer.tokenize((text))
  return text

def preprocess_trg_out(text):
  text = tokenizer.tokenize((text)) + ['<eos>']
  return text

토큰화 함수를 활용해 기존 데이프레임에 토큰화 한 컬럼을 추가합니다.

In [ ]:
train_data['src_in'] = train_data['Q'].apply(lambda x : preprocess_src(x))
train_data['trg_in'] = train_data['A'].apply(lambda x : preprocess_trg_in(x))
train_data['trg_out'] = train_data['A'].apply(lambda x : preprocess_trg_out(x))

In [ ]:
train_data

,Q,A,label,src_in,trg_in,trg_out
0,12시 땡!,하루가 또 가네요.,0,"[12, ##시, [UNK], !]","[<sos>, 하, ##루, ##가, 또, 가, ##네, ##요, .]","[하, ##루, ##가, 또, 가, ##네, ##요, ., <eos>]"
1,1지망 학교 떨어졌어,위로해 드립니다.,0,"[1, ##지, ##망, 학, ##교, 떨, ##어, ##졌, ##어]","[<sos>, 위, ##로, ##해, 드, ##립, ##니다, .]","[위, ##로, ##해, 드, ##립, ##니다, ., <eos>]"
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0,"[3, ##박, ##4, ##일, 놀, ##러, ##가, ##고, 싶, ##다]","[<sos>, 여, ##행, ##은, 언, ##제, ##나, 좋, ##죠, .]","[여, ##행, ##은, 언, ##제, ##나, 좋, ##죠, ., <eos>]"
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0,"[3, ##박, ##4, ##일, 정도, 놀, ##러, ##가, ##고, 싶, ##다]","[<sos>, 여, ##행, ##은, 언, ##제, ##나, 좋, ##죠, .]","[여, ##행, ##은, 언, ##제, ##나, 좋, ##죠, ., <eos>]"
4,PPL 심하네,눈살이 찌푸려지죠.,0,"[PP, ##L, 심, ##하, ##네]","[<sos>, 눈, ##살, ##이, 찌, ##푸, ##려, ##지, ##죠, .]","[눈, ##살, ##이, 찌, ##푸, ##려, ##지, ##죠, ., <eos>]"
...,...,...,...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2,"[훔, ##쳐, ##보, ##는, 것, ##도, 눈, ##치, 보, ##임, .]","[<sos>, 티, ##가, 나, ##니, ##까, 눈, ##치가, 보, ##이는,...","[티, ##가, 나, ##니, ##까, 눈, ##치가, 보, ##이는, 거, ##죠..."
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2,"[훔, ##쳐, ##보, ##는, 것, ##도, 눈, ##치, 보, ##임, .]","[<sos>, 훔, ##쳐, ##보, ##는, 거, 티, ##나, ##나, ##봐,...","[훔, ##쳐, ##보, ##는, 거, 티, ##나, ##나, ##봐, ##요, ...."
11820,흑기사 해주는 짝남.,설렜겠어요.,2,"[흑, ##기, ##사, 해, ##주는, 짝, ##남, .]","[<sos>, [UNK], .]","[[UNK], ., <eos>]"
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2,"[힘, ##든, 연, ##애, 좋은, 연, ##애, ##라는, ##게, 무, ##슨...","[<sos>, 잘, 헤, ##어, ##질, 수, 있는, 사, ##이, 여, ##부,...","[잘, 헤, ##어, ##질, 수, 있는, 사, ##이, 여, ##부, ##인, 거..."


다음으로 단어 사전을 구축합니다.

번역에서는 원문 사전과 번역문 사전을 따로 준비했지만, 챗봇은 질문과 답변문이 모두 한국어이기 때문에 하나의 사전을 생성하겠습니다.

In [ ]:
def build_vocab(sents):
  word_list = []

  for sent in sents:
      for word in sent:
        word_list.append(word)

  # 각 단어별 등장 빈도를 계산하여 등장 빈도가 높은 순서로 정렬
  # Counter는 단어를 키(key), 등장 빈도를 값(value)으로 가지도록 반환
  word_counts = Counter(word_list)
  vocab = sorted(word_counts, key=word_counts.get, reverse=True)

  word_to_index = {}
  word_to_index['<PAD>'] = 0
  word_to_index['<UNK>'] = 1

  # 등장 빈도가 높은 단어일수록 낮은 정수를 부여
  for index, word in enumerate(vocab) :
    if word != '<PAD>':
        word_to_index[word] = index + 2

  return word_to_index

In [ ]:
# 데이터프레임 답변문의 입력과 정답을 모두 포함하여 단어집합 생성
src_input = train_data['src_in'].to_list()
trg_input = train_data['trg_in'].to_list()
trg_output = train_data['trg_out'].to_list()

vocab = build_vocab(src_input + trg_input + trg_output)

vocab_size = len(vocab)
print(f"한국어 단어 집합의 크기 : {vocab_size}")

한국어 단어 집합의 크기 : 2210


In [ ]:
# 인코딩된 시퀀스를 단어로 역매핑하기 위한 딕셔너리 생성
index_to_tar = {v: k for k, v in vocab.items()}

단어사전 구축을 완료했습니다.

다음 작업은 문장의 길이를 일치시켜주는 작업과, 인코딩 작업을 진행해야 합니다.

우선 데이터의 문장 길이 경향을 파악해보겠습니다.

In [ ]:
# 각 문장의 길이(토큰 수)를 체크하여 새로운 컬럼으로 추가
train_data['src_in_len'] = train_data['src_in'].apply(len)
train_data['trg_in_len'] = train_data['trg_in'].apply(len)

In [ ]:
train_data

,Q,A,label,src_in,trg_in,trg_out,src_in_len,trg_in_len
0,12시 땡!,하루가 또 가네요.,0,"[12, ##시, [UNK], !]","[<sos>, 하, ##루, ##가, 또, 가, ##네, ##요, .]","[하, ##루, ##가, 또, 가, ##네, ##요, ., <eos>]",4,9
1,1지망 학교 떨어졌어,위로해 드립니다.,0,"[1, ##지, ##망, 학, ##교, 떨, ##어, ##졌, ##어]","[<sos>, 위, ##로, ##해, 드, ##립, ##니다, .]","[위, ##로, ##해, 드, ##립, ##니다, ., <eos>]",9,8
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0,"[3, ##박, ##4, ##일, 놀, ##러, ##가, ##고, 싶, ##다]","[<sos>, 여, ##행, ##은, 언, ##제, ##나, 좋, ##죠, .]","[여, ##행, ##은, 언, ##제, ##나, 좋, ##죠, ., <eos>]",10,10
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0,"[3, ##박, ##4, ##일, 정도, 놀, ##러, ##가, ##고, 싶, ##다]","[<sos>, 여, ##행, ##은, 언, ##제, ##나, 좋, ##죠, .]","[여, ##행, ##은, 언, ##제, ##나, 좋, ##죠, ., <eos>]",11,10
4,PPL 심하네,눈살이 찌푸려지죠.,0,"[PP, ##L, 심, ##하, ##네]","[<sos>, 눈, ##살, ##이, 찌, ##푸, ##려, ##지, ##죠, .]","[눈, ##살, ##이, 찌, ##푸, ##려, ##지, ##죠, ., <eos>]",5,10
...,...,...,...,...,...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2,"[훔, ##쳐, ##보, ##는, 것, ##도, 눈, ##치, 보, ##임, .]","[<sos>, 티, ##가, 나, ##니, ##까, 눈, ##치가, 보, ##이는,...","[티, ##가, 나, ##니, ##까, 눈, ##치가, 보, ##이는, 거, ##죠...",11,13
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2,"[훔, ##쳐, ##보, ##는, 것, ##도, 눈, ##치, 보, ##임, .]","[<sos>, 훔, ##쳐, ##보, ##는, 거, 티, ##나, ##나, ##봐,...","[훔, ##쳐, ##보, ##는, 거, 티, ##나, ##나, ##봐, ##요, ....",11,12
11820,흑기사 해주는 짝남.,설렜겠어요.,2,"[흑, ##기, ##사, 해, ##주는, 짝, ##남, .]","[<sos>, [UNK], .]","[[UNK], ., <eos>]",8,3
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2,"[힘, ##든, 연, ##애, 좋은, 연, ##애, ##라는, ##게, 무, ##슨...","[<sos>, 잘, 헤, ##어, ##질, 수, 있는, 사, ##이, 여, ##부,...","[잘, 헤, ##어, ##질, 수, 있는, 사, ##이, 여, ##부, ##인, 거...",16,17


In [ ]:
# 각 질문의 토큰 길이에 대한 집계함수 출력
train_data['src_in_len'].describe()

,src_in_len
count,11823.000000
mean,9.130847
std,4.056138
min,1.000000
25%,6.000000
50%,8.000000
75%,11.000000
max,37.000000


평균 길이가 9개 토큰 정도를 가지고 있는 문장이고, 가장 긴 문장은 37개의 토큰을 가지고 있습니다.

이는 데이터에서 평균 길이와 비교해서 지나치게 긴 문장들이 소수 포함되어 있다는 것을 의미합니다.

해당 문장 데이터를 제거함으로써, 길이가 어느정도 비슷한 데이터만 남도록 걸러주는 작업을 진행하겠습니다.

In [ ]:
# src_in_len이 15보다 큰 행 제거
train_data = train_data[train_data['src_in_len'] <= 15].reset_index(drop=True)

train_data

,Q,A,label,src_in,trg_in,trg_out,src_in_len,trg_in_len
0,12시 땡!,하루가 또 가네요.,0,"[12, ##시, [UNK], !]","[<sos>, 하, ##루, ##가, 또, 가, ##네, ##요, .]","[하, ##루, ##가, 또, 가, ##네, ##요, ., <eos>]",4,9
1,1지망 학교 떨어졌어,위로해 드립니다.,0,"[1, ##지, ##망, 학, ##교, 떨, ##어, ##졌, ##어]","[<sos>, 위, ##로, ##해, 드, ##립, ##니다, .]","[위, ##로, ##해, 드, ##립, ##니다, ., <eos>]",9,8
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0,"[3, ##박, ##4, ##일, 놀, ##러, ##가, ##고, 싶, ##다]","[<sos>, 여, ##행, ##은, 언, ##제, ##나, 좋, ##죠, .]","[여, ##행, ##은, 언, ##제, ##나, 좋, ##죠, ., <eos>]",10,10
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0,"[3, ##박, ##4, ##일, 정도, 놀, ##러, ##가, ##고, 싶, ##다]","[<sos>, 여, ##행, ##은, 언, ##제, ##나, 좋, ##죠, .]","[여, ##행, ##은, 언, ##제, ##나, 좋, ##죠, ., <eos>]",11,10
4,PPL 심하네,눈살이 찌푸려지죠.,0,"[PP, ##L, 심, ##하, ##네]","[<sos>, 눈, ##살, ##이, 찌, ##푸, ##려, ##지, ##죠, .]","[눈, ##살, ##이, 찌, ##푸, ##려, ##지, ##죠, ., <eos>]",5,10
...,...,...,...,...,...,...,...,...
10891,후회 없이 사랑하고 싶어,진심으로 다가가 보세요.,2,"[후, ##회, 없이, 사, ##랑, ##하고, 싶, ##어]","[<sos>, 진, ##심, ##으로, 다, ##가, ##가, 보, ##세, ##요...","[진, ##심, ##으로, 다, ##가, ##가, 보, ##세, ##요, ., <e...",8,11
10892,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2,"[훔, ##쳐, ##보, ##는, 것, ##도, 눈, ##치, 보, ##임, .]","[<sos>, 티, ##가, 나, ##니, ##까, 눈, ##치가, 보, ##이는,...","[티, ##가, 나, ##니, ##까, 눈, ##치가, 보, ##이는, 거, ##죠...",11,13
10893,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2,"[훔, ##쳐, ##보, ##는, 것, ##도, 눈, ##치, 보, ##임, .]","[<sos>, 훔, ##쳐, ##보, ##는, 거, 티, ##나, ##나, ##봐,...","[훔, ##쳐, ##보, ##는, 거, 티, ##나, ##나, ##봐, ##요, ....",11,12
10894,흑기사 해주는 짝남.,설렜겠어요.,2,"[흑, ##기, ##사, 해, ##주는, 짝, ##남, .]","[<sos>, [UNK], .]","[[UNK], ., <eos>]",8,3


이제 질문과 번역문을 각각 최대 길이로 만들어주는 작업을 진행하겠습니다.


예를 들어 질문들 중 가장 긴 문장이 16개의 토큰을 가진 문장이라면, 16개가 되지 않는 문장드에 패딩토큰을 추가해줌으로써 마찬가지로 길이를 16으로 만들어줍니다.

In [ ]:
# 질문과 답변문 최대 길이 확인

src_length= int(train_data['src_in_len'].max())
trg_length= int(train_data['trg_in_len'].max())
src_length, trg_length

(15, 45)

In [ ]:
# 질문 패딩 및 인코딩
def src_encoding(tokens):
    # (src_length - 토큰 수) 만큼 패딩토큰 추가 - 패딩토큰을 추가하여 최대길이로 맞춤
    tokens = tokens + ['<PAD>'] *  (src_length - len(tokens))

    # 인코딩 된 숫자를 담아 둘 리스트
    index_sequences = []

    # 문장에서 토큰을 꺼내오며
    for word in tokens:
      try: # 토큰 인코딩 (단어 사전에 없는 토큰이 들어오면 except로)
          index_sequences.append(vocab[word])
      except KeyError: # 단어 사전에 없는 토큰이 들어오면 '<UNK>' 토큰의 숫자로 변환
          index_sequences.append(vocab['<UNK>'])

    return index_sequences

In [ ]:
# 답변문 패딩 및 인코딩
def trg_encoding(tokens):
    # (src_length - 토큰 수) 만큼 패딩토큰 추가
    tokens = tokens + ['<PAD>'] *  (trg_length - len(tokens))

    # 인코딩 된 숫자를 담아 둘 리스트
    index_sequences = []

    # 문장에서 토큰을 꺼내오며
    for word in tokens:
      try: # 토큰 인코딩 (단어 사전에 없는 토큰이 들어오면 except로)
          index_sequences.append(vocab[word])
      except KeyError: # 단어 사전에 없는 토큰이 들어오면 '<UNK>' 토큰의 숫자로 변환
          index_sequences.append(vocab['<UNK>'])

    return index_sequences

In [ ]:
# 데이터프레임의 질문과 답변문 각각에 패딩 및 인코딩 함수 적용
train_data['src_in'] = train_data['src_in'].apply(lambda x : src_encoding(x))
train_data['trg_in'] = train_data['trg_in'].apply(lambda x : trg_encoding(x))
train_data['trg_out'] = train_data['trg_out'].apply(lambda x : trg_encoding(x))

In [ ]:
train_data

,Q,A,label,src_in,trg_in,trg_out,src_in_len,trg_in_len
0,12시 땡!,하루가 또 가네요.,0,"[1886, 111, 26, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[4, 66, 334, 11, 318, 48, 19, 3, 2, 0, 0, 0, 0...","[66, 334, 11, 318, 48, 19, 3, 2, 5, 0, 0, 0, 0...",4,9
1,1지망 학교 떨어졌어,위로해 드립니다.,0,"[608, 13, 590, 512, 469, 458, 8, 260, 8, 0, 0,...","[4, 394, 103, 10, 172, 957, 72, 2, 0, 0, 0, 0,...","[394, 103, 10, 172, 957, 72, 2, 5, 0, 0, 0, 0,...",9,8
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0,"[678, 801, 1708, 86, 297, 122, 11, 23, 88, 51,...","[4, 190, 259, 29, 283, 114, 20, 18, 35, 2, 0, ...","[190, 259, 29, 283, 114, 20, 18, 35, 2, 5, 0, ...",10,10
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0,"[678, 801, 1708, 86, 809, 297, 122, 11, 23, 88...","[4, 190, 259, 29, 283, 114, 20, 18, 35, 2, 0, ...","[190, 259, 29, 283, 114, 20, 18, 35, 2, 5, 0, ...",11,10
4,PPL 심하네,눈살이 찌푸려지죠.,0,"[2051, 2052, 367, 36, 19, 0, 0, 0, 0, 0, 0, 0,...","[4, 264, 850, 7, 1174, 1465, 60, 13, 35, 2, 0,...","[264, 850, 7, 1174, 1465, 60, 13, 35, 2, 5, 0,...",5,10
...,...,...,...,...,...,...,...,...
10891,후회 없이 사랑하고 싶어,진심으로 다가가 보세요.,2,"[167, 207, 887, 16, 30, 105, 88, 8, 0, 0, 0, 0...","[4, 197, 137, 148, 93, 11, 11, 90, 6, 3, 2, 0,...","[197, 137, 148, 93, 11, 11, 90, 6, 3, 2, 5, 0,...",8,11
10892,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2,"[1689, 524, 12, 22, 69, 15, 264, 322, 90, 687,...","[4, 736, 11, 43, 59, 33, 264, 978, 90, 600, 17...","[736, 11, 43, 59, 33, 264, 978, 90, 600, 17, 3...",11,13
10893,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2,"[1689, 524, 12, 22, 69, 15, 264, 322, 90, 687,...","[4, 1689, 524, 12, 22, 17, 736, 20, 20, 40, 3,...","[1689, 524, 12, 22, 17, 736, 20, 20, 40, 3, 2,...",11,12
10894,흑기사 해주는 짝남.,설렜겠어요.,2,"[1473, 52, 132, 44, 566, 138, 204, 2, 0, 0, 0,...","[4, 26, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[26, 2, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",8,3


데이터에 패딩과 인코딩을 완료했습니다.

이제 데이터를 훈련용 데이터와 검증용 데이터로 나눠준 후, 데이터셋과 데이터로더를 생성하도록 하겠습니다.

In [ ]:
from sklearn.model_selection import train_test_split

# 데이터프레임에서 무작위로 80%를 훈련 데이터로, 20%를 검증 데이터로 분할
train_df, valid_df = train_test_split(train_data, test_size=0.2, random_state=42)

# 결과 확인
print(f"훈련 데이터 크기: {len(train_df)}")
print(f"검증 데이터 크기: {len(valid_df)}")

훈련 데이터 크기: 8716
검증 데이터 크기: 2180


In [ ]:
# 데이터셋 클래스 생성 -> 데이터프레임에서 전처리된 데이터를 가져와서 텐서로 변환

import torch
from torch.utils.data import Dataset, DataLoader

class ChatbotDataset(Dataset):
    def __init__(self, dataframe):
        # df로부터 원문과 번역문을 받아옴
        self.src_in = dataframe['src_in'].values
        self.trg_in = dataframe['trg_in'].values
        self.trg_out = dataframe['trg_out'].values

    def __len__(self):
        return len(self.src_in)

    def __getitem__(self, idx):
        # 텐서 형태로 변경(데이터 타입은 정수)
        src_seq = torch.tensor(self.src_in[idx], dtype=torch.long)
        trg_in_seq = torch.tensor(self.trg_in[idx], dtype=torch.long)
        trg_out_seq = torch.tensor(self.trg_out[idx], dtype=torch.long)

        return src_seq, trg_in_seq, trg_out_seq # 질문 입력, 답변 입력, 답변 출력

In [ ]:
# 생성한 데이터셋 클래스를 활용해 데이터셋과 데이터로더 객체 생성

BATCH_SIZE = 256

# 훈련 데이터셋 및 데이터로더
train_dataset = ChatbotDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# 검증 데이터셋 및 데이터로더
valid_dataset = ChatbotDataset(valid_df)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
train_dataset[0]

(tensor([660, 766,  42,  76, 427,  81, 212, 188, 108,  11,  87,  19,   0,   0,
           0]),
 tensor([  4, 660, 766,   7,  30, 212, 188,  29,  93, 187,  59,  33,   3,   2,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0]),
 tensor([660, 766,   7,  30, 212, 188,  29,  93, 187,  59,  33,   3,   2,   5,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0]))

In [ ]:
batch_sample = next(iter(train_loader))

print('src in shape :', batch_sample[0].shape)
print('trg in shape :', batch_sample[1].shape)
print('trg out shape :', batch_sample[2].shape)

src in shape : torch.Size([256, 15])
trg in shape : torch.Size([256, 45])
trg out shape : torch.Size([256, 45])


In [ ]:
device

device(type='cuda')

### 2) 모델 구축


이제 전처리된 데이터를 학습시키기 위한 모델을 설계합니다. 모델은 트랜스포머를 기반으로 한 seq2seq 모델을 구축하겠습니다.

파이토치의 공식 문서를 참조했습니다.
https://pytorch.org/tutorials/beginner/translation_transformer.html



트랜스포머 모델은 다음과 같은 순서로 모델을 구축해야 합니다.


- 포지셔널 인코딩 클래스
- 마스킹 함수
- 모델 설계

우선 포지셔널 인코딩 함수를 구현하겠습니다.



In [ ]:
# 수학 연산을 위한 math 라이브러리
import math

class PositionalEncoding(nn.Module):
    # 클래스 생성 시 emb_size, dropout, maxlen을 입력으로 받음
    def __init__(self, emb_size, dropout, maxlen=5000):
        super(PositionalEncoding, self).__init__()

        # (maxlen * emb_size)크기의 PostionalEncoding 행렬 생성 후 원본과 타겟입력값에 더해줌

        # den :10000^(2i/d_model) 구현
        den = 10000 ** (torch.arange(0, emb_size, 2) / emb_size)

        # position
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)

        # 영 행렬 생성(maxlen, emb_size)
        pos_embedding = torch.zeros((maxlen, emb_size))

        # 영 행렬의 짝수 열은 사인함수 적용(0,2,4,6 .....)
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        # 영 행렬의 홀수 열은 코사인 함수 적용(1,3,5,7)
        pos_embedding[:, 1::2] = torch.cos(pos * den)

        # pistional embedding에 배치 차원 추가(1, maxlen, embsize)
        pos_embedding = pos_embedding.unsqueeze(0)

        # 논문에서는 포지셔널인코딩을 거친 후 드롭아웃을 적용
        self.dropout = nn.Dropout(dropout)

        # PyTorch의 nn.Module에서 제공하는 기능으로, 모델의 상태에 포함되지만 학습되지 않는 텐서를 등록할 때 사용
        # PositionalEncoding 벡터는 변하지 않고 고정
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding):
        # 임베딩 벡터와(batch, seq_length, embsize) PE벡터(1, maxlen, embsize) 더함
        # 배치의 각 데이터들에대해 각각 PE 더함
        # 이후 드롭아웃 적용
        return self.dropout(token_embedding + self.pos_embedding[:, :token_embedding.size(1), :])

다음으로 모델을 설계하겠습니다. 모델은 입력데이터(원문,번역문)가 임베딩 후 포지셔널 인코딩을 적용한 뒤 트랜스포머를 통과합니다. 이 과정에서 입력값과 정답값의 패딩은 무시될 수 있도록 패딩마스크를 넣어주고, 디코더에서 self-attention 시 미래 시점의 값을 참조할 수 없도록 미래 시점 마스크(look-ahead-mask)를 함께 인자로 넣어줍니다.

트랜스포머를 통과한 후 출력 레이어를 단어사전 개수 만큼의 출력값이 생성됩니다.(가장 큰 값을 가진 인덱스의 단어로 예측)

<br>

또한 예측 과정에서 사용할 인코딩, 디코딩 작업을 위한 함수와 패딩마스크, 미래시점 마스크를 위한 함수를 생성합니다.

In [ ]:
class Seq2SeqTransformer(nn.Module):
    def __init__(self, num_encoder_layers, num_decoder_layers, emb_size,
                 nhead, src_vocab_size, tgt_vocab_size, dim_feedforward=512,
                 dropout=0.1):
        super(Seq2SeqTransformer, self).__init__()
        # 모델에서 사용할 레이어 정의

        # 임베딩 레이어
        self.emb_size = emb_size
        self.embedding = nn.Embedding(vocab_size, emb_size)

        # 포지셔널 인코딩 레이어
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

        # 트랜스포머 레이어(인코더+디코더)
        # d_model : 입출력 차원 수, num_encoder(decoder)_layers : 인코더(디코더)의 층 수)
        # dim_feedforward : FFNN의 차원 수(attention 거친 후 FFNN에서 dmodel -> dim_feedforward -> dmodel)
        self.transformer = nn.Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout,
                                       batch_first=True)

        # 출력 레이어(단어사전에 있는 단어로 출력될 수 있도록)
        self.fc = nn.Linear(emb_size, tgt_vocab_size)

    def forward(self, src, tgt, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, memory_key_padding_mask):
        # 질문, 답변문 임베딩
        # 논문에서는 임베딩 벡터의 스케일을 조정하여 초기화 과정에서의 불안정성을 줄이기 위해 임베딩 크기의 제곱근을 곱함
        src, tgt = src.long(), tgt.long()
        src = self.embedding(src) * math.sqrt(self.emb_size)
        tgt = self.embedding(tgt) * math.sqrt(self.emb_size)

        # 포지셔널 인코딩 적용
        src_emb = self.positional_encoding(src)
        tgt_emb = self.positional_encoding(tgt)

        # 트랜스포머 모델 통과
        # src_emb : 질문 데이터, tgt_emb : 답변문 데이터, src_mask : 질문 마스크, tgt_mask : 답변문 마스크(어텐션 시 미래 시점 못보도록)
        # src(tgt)_padding_mask : 질문(답변문)이 어텐션 시 패딩토큰은 적용되지 않도록 패딩 토큰에 마스킹-> self-atteniton에 적용
        # memory_key_padding_mask : 디코더가 인코더의 출력 정보 활용 시 패딩된 부분 활용하지 않도록 마스 -> encoder-decoder attention에 적용
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None, # None 부분은 memory 마스크로 보통 None으로 둠
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        outs = self.fc(outs)
        return outs # (배치사이즈, 출력시퀀스 길이, 단어 사전 수)


    # 예측 시 적용할 인코딩 함수
    def encode(self, src, src_mask):
        # 예측 시 사용할 인코더 통과(질문 -> 임베딩 -> 포지셔널인코딩 -> 트랜스포머 인코더 통과)
        src = src.long()
        src = self.embedding(src) * math.sqrt(self.emb_size)
        src_emb = self.positional_encoding(src)

        # 실제 예측 시 패딩을 넣지 않기에 패딩 마스크는 적용 X
        outs = self.transformer.encoder(src_emb, src_mask)
        return outs

    # 예측 시 적용할 디코딩 함수
    def decode(self, tgt, memory, tgt_mask):
        # 에측 시 디코더 통과(답변문 -> 임베딩 -> 포지셔널인코딩 -> 트랜스포머 디코더 통과)
        # 예측 시 tgt는 sos 토큰, memory는 인코더의 출력(매 층에서 인코더 디코더 어텐션에 활용)
        tgt = tgt.long()
        tgt = self.embedding(tgt) * math.sqrt(self.emb_size)
        tgt_emb = self.positional_encoding(tgt)

        outs = self.transformer.decoder(tgt_emb, memory, tgt_mask)
        return outs

# 미래 시점 마스크를 위한 함수 생성
def generate_square_subsequent_mask(sz):
    # torch.ones((sz, sz) : (sz, sz)크기의 1로 채워진 행렬 생성
    # torch.triu() : 대각선 기준으로 아래의 원소를 0으로 변경
    # 1 1 1
    # 0 1 1
    # 0 0 1

    # 전치
    # 1 0 0
    # 1 1 0
    # 1 1 1
    mask = torch.triu(torch.ones((sz, sz), device=device)).transpose(0, 1)

    # 0인 부분을 음의 무한 값으로, 1인 부분을 0.0으로 변경
    # 타켓문의 시퀀스 길이 만큼의 마스크 행렬을 생성하여 self-attention 시 미래 시점의 값들은 음의 무한대로 갈 수 있도록
    # 음의 무한대 값은 softmax 통과 시 0으로 수렴
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

# 전체 마스크(패딩, 미래시점)를 생성하기 위한 함수
def create_mask(src, tgt):
    # 질문과 답변문의 시퀀스 길이 확인
    src_seq_len = src.shape[1]
    tgt_seq_len = tgt.shape[1]

    # 미래시점 마스크
    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)

    # 질문은 마스킹을 적용하지 않기에 False로 이루어진 행렬 생성
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=device).type(torch.bool)

    # 패딩마스크 생성(입력 데이터 행렬에서 입력 토큰이 0(<PAD>)인 부분을 True로 나머지는 False인 형태로 반환)
    src_padding_mask = (src == 0)
    tgt_padding_mask = (tgt == 0)

    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask


모델 구축을 완료했습니다.

### 3) 모델 하이퍼파라미터 설정

다음으로 모델의 하이퍼파라미터를 설정합니다.

- 손실 함수
- 옵티마이저
- 트랜스포머 하이퍼파라미터

In [ ]:
torch.manual_seed(0)

# 질문과 답변문의 단어사전 수(출력사이즈)
# 질문과 답변문이 같은 언어이므로 동일한 단어 사전 사용
SRC_VOCAB_SIZE = vocab_size
TGT_VOCAB_SIZE = vocab_size
EMB_SIZE = 512 # 단어의 임베딩 사이즈이자, 트랜스포머 입출력 크기
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3


# Early stopping 설정
patience = 5  # 얼리스탑핑을 위한 인내 횟수
early_stopping_counter = 0


# Training loop
best_val_loss = 99999

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

transformer = transformer.to(device)

# 패딩은 loss 계산 시 적용하지 않음
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=0)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)


### 4) 모델 학습


이제 모델을 학습시킬 준비가 완료되었습니다.
학습을 위한 함수와, 검증을 위한 함수를 생성하고 매 epoch 시 두 함수를 실행하여 학습과 검증 작업을 진행합니다.


추가로 손실과 정확도 계산 시, 정답 시퀀스의 각 단어와 모델이 예측한 시퀀스의 각 단어별로 loss와 정확도를 계산해야 하기 때문에 둘을 펼쳐 놓고 각각 계산하도록 구현합니다.


ex.
- 정답문장 : [2, 2, 3, 1, 0]
- 예측문장 : [2, 2, 3, 0, 4]
- 기존방식에서는 문제가 1개이고 정답데이터와 예측 데이터가 다르므로 오답이기에 정확도가 0이지만, 여기서는 안의 단어 하나씩 비교하기 때문에 문제가 5개가 되고 맞춘 개수가 3개이기에 정확도 0.6

In [ ]:
def train_epoch(model, optimizer):
    model.train()
    losses = 0 # 각 배치의 손실값 누적시킬 변수
    total_correct = 0 # 정확도 계산을 위해 맞춘 개수 누적
    total_count = 0 # 전체 문제 수 계산을 위해 배치 당 문제 개수 누적


    for src, tgt_input, tgt_out in train_loader:
        src = src.to(device)
        tgt_input = tgt_input.to(device)
        tgt_out = tgt_out.to(device)

        # 위에서 만든 마스킹 함수를 통해 마스크 생성
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        # 모델에 원문, 답변문, 마스크를 넣어 순전파
        logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)

        # 기울기 초기화
        optimizer.zero_grad()

        # 손실 계산(문장이 아닌 시퀀스를 펼쳐서 각 단어에 대해 계산)
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward() #역전파

        optimizer.step() # Weight bias 업데이트
        losses += loss.item() # 손실값 기록(배치 평균 손실)

        # 정확도 계산 시 패딩 부분은 제외
        mask = (tgt_out != 0) # 패딩이 아닌 부분만 True, 나머지는 False로

        # 정답과 예측값을 비교하여 맞춘 개수 계산 후, 마스크를 곱하여 True인 부분(패딩이 아닌 토큰)만 개수 합쳐서 기록
        total_correct += ((logits.argmax(dim=-1) == tgt_out) * mask).sum().item()

        # 마스크에서 True인 부분만 개수 계산하여 기록
        total_count += mask.sum().item()

    # 손실값은 배치별 평균이 누적되었기에 배치의 수(train_loader의 수)로 나눔
    # 정확도는 맞춘개수가 누적되었기에 전체 문제의 수로 나눔
    return losses / len(train_loader), total_correct / total_count


# model.eval(), 검증 데이터 사용을 제외하고 학습 함수와 동일
def evaluate(model):
    model.eval()
    losses = 0
    total_correct = 0
    total_count = 0

    for src, tgt_input, tgt_out in valid_loader:
        src = src.to(device)
        tgt_input = tgt_input.to(device)
        tgt_out = tgt_out.to(device)

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)

        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

        mask = (tgt_out != 0)
        total_correct += ((logits.argmax(dim=-1) == tgt_out) * mask).sum().item()
        total_count += mask.sum().item()

    return losses / len(valid_loader), total_correct / total_count

In [ ]:
# 시간 기록을 편하게 하기 위한 라이브러리
from timeit import default_timer as timer

# 학습할 에폭 수 설정
NUM_EPOCHS = 1000

for epoch in range(1, NUM_EPOCHS+1):
    # epoch을 돌며 학습 함수와, 검증 함수 실행

    # 시간 기록 시작
    start_time = timer()

    # 학습함수 실행 -> Train loss와 정확도 반환
    train_loss, train_acc = train_epoch(transformer, optimizer)

    # 학습 종료 시간 기록(1epoch 당 학습 시간)
    end_time = timer()

    # 검증함수 실행 -> valid loss와 valid 정확도 반환
    val_loss, val_acc  = evaluate(transformer)
    print(f'Epoch: {epoch} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Valid Loss: {val_loss:.4f} | Valid Acc: {val_acc:.4f}')

    # 검증 손실이 최소일 때 체크포인트 저장
    if val_loss < best_val_loss:
        print(f'Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. 체크포인트를 저장합니다.')
        best_val_loss = val_loss
        torch.save(transformer.state_dict(), 'best_model_checkpoint.pth')
        early_stopping_counter = 0  # 개선되었으므로 카운터 초기화
    else:
        early_stopping_counter += 1  # 개선되지 않으면 카운터 증가

    # 얼리스탑핑 조건 확인 후 작동
    if early_stopping_counter >= patience:
        print(f'Validation loss가 {patience}번의 에폭 동안 개선되지 않았습니다. 학습을 조기 종료합니다.')
        break

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch: 1 | Train Loss: 5.8978 | Train Acc: 0.1908 | Valid Loss: 5.0018 | Valid Acc: 0.2283
Validation loss improved from 99999.0000 to 5.0018. 체크포인트를 저장합니다.
Epoch: 2 | Train Loss: 4.8214 | Train Acc: 0.2565 | Valid Loss: 4.5230 | Valid Acc: 0.2901
Validation loss improved from 5.0018 to 4.5230. 체크포인트를 저장합니다.
Epoch: 3 | Train Loss: 4.4805 | Train Acc: 0.2972 | Valid Loss: 4.2366 | Valid Acc: 0.3288
Validation loss improved from 4.5230 to 4.2366. 체크포인트를 저장합니다.
Epoch: 4 | Train Loss: 4.2290 | Train Acc: 0.3292 | Valid Loss: 3.9997 | Valid Acc: 0.3486
Validation loss improved from 4.2366 to 3.9997. 체크포인트를 저장합니다.
Epoch: 5 | Train Loss: 4.0125 | Train Acc: 0.3515 | Valid Loss: 3.8015 | Valid Acc: 0.3709
Validation loss improved from 3.9997 to 3.8015. 체크포인트를 저장합니다.
Epoch: 6 | Train Loss: 3.8148 | Train Acc: 0.3701 | Valid Loss: 3.6648 | Valid Acc: 0.3793
Validation loss improved from 3.8015 to 3.6648. 체크포인트를 저장합니다.
Epoch: 7 | Train Loss: 3.6578 | Train Acc: 0.3848 | Valid Loss: 3.5018 | Valid

학습을 완료했습니다.

### 5) 학습한 모델을 활용한 예측

이제 학습한 모델을 활용해 실제로 사용하기 위한 함수들을 작성합니다.

미리 만들어둔 인코딩 디코딩 함수를 활용하고, 디코더는 for문을 통해 종료토큰이 나오거나, 종료 토큰이 나오지 않는다면 최대로 설정해둔 길이만큼 결과를 출력하도록 반복해서 작동시킵니다.


매 반복에서 나온 값들을 누적해가며 다음 시점의 입력으로 사용하고, 반복이 멈추는 시점이라면 현재까지 누적한 결과들을 리턴해줍니다.

<br>

또한 후처리 함수를 통해, 토큰화 과정에서 발생한 '#"등의 처리를 진행해줍니다.

In [ ]:
import re


# 예측 함수 작성
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    # 예측할 질문
    src = src.to(device)

    # 예측할 질문 마스크
    src_mask = src_mask.to(device)

    # 미리 작성한 인코더 함수를 통해 인코더 통과 값 반환
    memory = model.encode(src, src_mask)

    # start_symbol을 받아서 디코더에 넣어 줄 <sos>에 대한 입력값 생성
    # 배치1, 시퀀스 길이 1 사이즈 (1,1)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)

    # 최대로 예측할 문장길이만큼 for문(max_len은 앞에서 작성한 postional-encoding 행렬과 동일)
    for i in range(max_len - 1):

        # 인코더 출력값
        memory = memory.to(device)

        # 미래시점 마스크 생성
        tgt_mask = generate_square_subsequent_mask(ys.size(1)).to(device)

        # 미리 생성한 디코딩 함수 작동
        out = model.decode(ys, memory, tgt_mask) # 출력 사이즈 : (배치, 시퀀스길이, dmodel)

        # 매 반복마다 마지막 타임스텝의 예측만 사용
        # 예를 들어 첫 반복에 '[나는]', 두번째 반복에 '[나는, 내일]'세번째 반복에 '[나는, 내일, 학교에]'와 같이 예측하기 때문에 매 반복마다 마지막 시점만 가지고 오도록
        prob = model.fc(out[:, -1])
        _, next_word = torch.max(prob, dim=1) # argmax용법과 동일-> 가장 큰 값의 인덱스 반환
        next_word = next_word.item() # 모델의 예측 클래스

        # 기존 입력에 예측한 문장 추가 후 다음 반복에서 입력으로 활용
        # ex. 첫 루프 후 디코더 입력 데이터는 '<sos>, 나는', 두번째 루프 후 '<sos>', '나는', '내일' ....
        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
        if next_word == 5:  # EOS 토큰이 나왔다면 다은 루프를 돌지 않고 종료
            break
    return ys # 루프 종료 후의 최종 예측 단어들



# 위의 예측 함수에 후처리 함수를 추가해 최종 번역 함수 작성
def translate(model, input_seq):
    # 모델과 입력 시퀀스를 받아와, 모델을 검증 모드로
    model.eval()

    # 입력 시퀀스를 텐서 형태로 변환
    encoder_inputs = torch.tensor(input_seq, dtype=torch.long).unsqueeze(0).to(device)

    # 시퀀스 길이 확인
    num_tokens = encoder_inputs.shape[1]

    # 입력 데이터 마스크 작성(질문은 패딩마스크를 적용하지 않기에 False로 채워진 값)
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)

    # 위에 작성한 예측 함수 작동
    tgt_tokens = greedy_decode(
        model,  encoder_inputs, src_mask, max_len=num_tokens + 5, start_symbol=4).flatten()

    # tgt_tokens을 역매핑용 단어사전(위에서 만들어둠)을 사용해 디코딩
    translated_tokens = [index_to_tar[token.item()] for token in tgt_tokens if token.item() in index_to_tar]

    # 서브워드 토큰 처리: '#'이 포함된 경우 이전 단어와 결합
    translated_sentence = " ".join(translated_tokens)
    translated_sentence = re.sub(r'\s*##', '', translated_sentence)  # ## 제거하고 앞 단어와 붙임

    # 마침표와 앞 단어를 붙이기
    translated_sentence = re.sub(r'\s+([.,!?])', r'\1', translated_sentence)

    # <sos>와 <eos> 토큰을 제거
    translated_sentence = translated_sentence.replace("<sos>", "").replace("<eos>", "").strip()

    return translated_sentence

In [ ]:
# 검증용 데이터프레임에서 3, 50, 100, 1001번 인덱스의 값들을 예측

for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = valid_df['src_in'].iloc[seq_index]
  translated_text = translate(transformer, input_seq)

  print("질문 :",valid_df['Q'].iloc[seq_index])
  print("답변 :",translated_text)
  print("-"*50)

질문 : 자격증 공부해야지
답변 : 소원을 비세요.
--------------------------------------------------
질문 : 언제쯤 잊혀질까
답변 : 기억에서 울어도 괜찮아요.
--------------------------------------------------
질문 : 할 줄 아는거 뭐야?
답변 : 저도 보고 싶어요.
--------------------------------------------------
질문 : 헤어지기 전으로 돌아가고 싶어.
답변 : 마음이 복잡한가봐요.
--------------------------------------------------
질문 : 환승이 될까
답변 : 저도 보고 싶어요.
--------------------------------------------------
